In [1]:
def ip_to_binary(ip_address: str) -> str:
    octets = ip_address.split('.')
    binary_octets = []
    
    for octet in octets:
        binary = bin(int(octet))[2:]
        binary_octets.append(binary.zfill(8))
    
    return ''.join(binary_octets)

In [2]:
def get_network_prefix(ip_cidr: str) -> str:
    ip_part, prefix_length = ip_cidr.split('/')
    prefix_length = int(prefix_length)
    
    binary_ip = ip_to_binary(ip_part)
    network_prefix = binary_ip[:prefix_length]
    
    return network_prefix

In [3]:
class Router:
    def __init__(self, routes):
        self.routes = routes
        self.forwarding_table = []
        self.build_forwarding_table(routes)
    
    def build_forwarding_table(self, routes):
        for cidr, link in routes:
            binary_prefix = get_network_prefix(cidr)
            self.forwarding_table.append((binary_prefix, link))
        
        self.forwarding_table.sort(key=lambda x: len(x[0]), reverse=True)
    
    def route_packet(self, dest_ip: str) -> str:
        binary_dest = ip_to_binary(dest_ip)
        
        for prefix, link in self.forwarding_table:
            if binary_dest.startswith(prefix):
                return link
        
        return "Default Gateway"

In [4]:
routes = [
    ("223.1.1.0/24", "Link 0"),
    ("223.1.2.0/24", "Link 1"),
    ("223.1.3.0/24", "Link 2"),
    ("223.1.0.0/16", "Link 4 (ISP)")
]

router = Router(routes)

print("Testing Router:")
print(f"223.1.1.100 -> {router.route_packet('223.1.1.100')}")
print(f"223.1.2.5 -> {router.route_packet('223.1.2.5')}")
print(f"223.1.250.1 -> {router.route_packet('223.1.250.1')}")
print(f"198.51.100.1 -> {router.route_packet('198.51.100.1')}")

Testing Router:
223.1.1.100 -> Link 0
223.1.2.5 -> Link 1
223.1.250.1 -> Link 4 (ISP)
198.51.100.1 -> Default Gateway
